In [4]:
files

['ANN_TP.csv', 'KK_TP.csv', 'LEW_TP.csv']

In [4]:
"""
Here, the datafiles are imported from Plot_2b_NIsTidalPhase_Normalvs.ExtremeMagnitude_Station1_FileGenerator.ipynb
Read the instruction of this code very carefully!!!!!!!!!

Job: Takes in two processed tidal phase csv file for two stations.
ANN: Upstream station
KK: Downstream station

Compared their tidal phase interaction magnitude at HT, LT, FT, RT.

Here, we create a 4x6 plot. The first two rows will signify the coastal stations, 

"""
import time
import glob
import os
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import linregress
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

%matplotlib qt
directory_path = r'F:\ProcessedDataStation\2011_Irene\Plot_d'
files = os.listdir(directory_path)

dataframe_list = []

for file_name in files:
    file_path = os.path.join(directory_path, file_name)
    station_name=file_path.split('\\')[-1].split('.')[0].split('_')[0]
    
    data_df = pd.read_csv(file_path)
    data_df['Station'] = station_name
    dataframe_list.append(data_df)

# Concatenate all the DataFrames vertically
tidalphase_df= pd.concat(dataframe_list, axis=0)


# Mapping indices to tidal phases and color palettes

station_dict = {
    "SP": "Sewells Point, VA",
    "MP": "Money Point, VA",
    "KK": "Kiptopeke, VA",
    "WAC": "Wachapreague, VA",
    "YOR": "Yorktown USCG, VA",
    "WP": "Windmill Point, VA",
    "OC": "Ocean City Inlet, MD",
    "LEW": "Lewisetta, VA",
    "SI": "Solomons Island, MD",
    "DC": "Washington, DC",
    "BH": "Bishops Head, MD",
    "CC": "Chesapeake City, MD",
    "BAL": "Baltimore, MD",
    "TB": "Tolchester Beach, MD",
    "ANN": "Annapolis, MD",
    "CAM": "Cambridge, MD",
    "DAH": "Dahlgren, VA",
    "CBBT": "CBBT, Chesapeake Channel, VA"
}

tidal_phase_mapping = {
    0: (['HT', 'LT'], "Set3"),
    1: (['RT', 'FT'], "vlag"),
}

# Mapping indices to stations and titles
station_mapping = {
    0: 'KK',
    1: 'KK',
    2: 'LEW',
    3: 'LEW',
    4: 'ANN',
    5: 'ANN',
}

# Create violin plots for each of the columns across tidal phases and stations in a 4x4 layout
col=6
fig, axes = plt.subplots(4, 6, figsize=(30, 20))


"""
This code is designed to iterate through a specified range of column indices, 
applying filters and visualization settings based on the current index and tidal phase data. 
For each index in the loop, the code first checks whether the index is even or odd. If the index is even, 
the code filters the tidalphase_df DataFrame to include only rows where the 
'Tidal Phase' is either 'HT' (High Tide) or 'LT' (Low Tide) and selects a color palette from seaborn using the
"magma" scheme, with the number of colors corresponding to the number of unique 'Station' values in the filtered data. 
If the index is odd, the code filters for 'RT' (Rising Tide) or 'FT' (Falling Tide) phases and selects a "Spectral" color palette.

After filtering by tidal phase, the code further refines the data based on the specific index value to focus on one of 
three stations: 'KK', 'LEW', or 'ANN'. For indices 0 and 1, the DataFrame is filtered to include only data from the 'KK' 
station, and the corresponding subplot is titled 'KK'. For indices 2 and 3, the code filters for the 'LEW' station and 
titles the subplot 'LEW'. Similarly, for indices 4 and 5, the DataFrame is filtered for the 'ANN' station, and the subplot 
is titled 'ANN'. The filtered data and selected color palette are then used for plotting in specific subplots, allowing for 
a customized visualization of tidal phases across different stations.

"""


# Loop through each column
for idx in range(col):
    # Determine tidal phases and color palette based on the index
    
    tidal_phases, palette_name = tidal_phase_mapping[idx % 2]
    color_palette = sns.color_palette(palette_name, n_colors=tidalphase_df['Station'].nunique())
    
    # Filter the DataFrame by the corresponding station and tidal phases
    station = station_mapping[idx]
    filtered_df = tidalphase_df[(tidalphase_df['Station'] == station) & 
                                (tidalphase_df['Tidal Amplitude'].isin(tidal_phases))]
    
    # Set the title for each column based on the station
    axes[0, idx].set_title(station_dict[station])

    # Populate the each column with the violin plots
    sns.violinplot(data=filtered_df, x='Labels', y='TSI+SS', hue='Tidal Amplitude', 
                   split=True, palette=color_palette, ax=axes[0, idx], inner='quartile')
    
    sns.violinplot(data=filtered_df, x='Labels', y='TRI+RF', hue='Tidal Amplitude', 
                   split=True, palette=color_palette, ax=axes[1, idx], inner='quartile')
    
    sns.violinplot(data=filtered_df, x='Labels', y='TWI+WV', hue='Tidal Amplitude', 
                   split=True, palette=color_palette, ax=axes[2, idx], inner='quartile')
    
    sns.violinplot(data=filtered_df, x='Labels', y='TSRWI', hue='Tidal Amplitude', 
                   split=True, palette=color_palette, ax=axes[3, idx], inner='quartile')



     # Set x-labels and y-labels with fontsize 12
    axes[0, idx].set_xlabel('', fontsize=16)  # Remove x-axis label
    axes[0, idx].set_ylabel('TSI+SS', fontsize=16)
    
    axes[1, idx].set_xlabel('', fontsize=16)  # Remove x-axis label
    axes[1, idx].set_ylabel('TRI+RF', fontsize=16)
    
    axes[2, idx].set_xlabel('', fontsize=16)  # Remove x-axis label
    axes[2, idx].set_ylabel('TWI+WV', fontsize=16)
    
    axes[3, idx].set_xlabel('', fontsize=16)  # Remove x-axis label
    axes[3, idx].set_ylabel('TSRWI', fontsize=16)
    
    # Set tick parameters with fontsize 12
    for ax in axes[:, idx]:
        ax.tick_params(axis='both', which='major', labelsize=16)

    # Set legend font size to 12
    for ax in axes[:, idx]:
        leg = ax.get_legend()
        if leg:
            leg.set_bbox_to_anchor((1, 1))  # Optional: adjust legend position
            leg.set_title(leg.get_title().get_text(), prop={'size': 16})
            plt.setp(leg.get_texts(), fontsize=16)



    

    axes[0, idx].set_xlabel('')  # Remove x-axis label
    axes[1, idx].set_xlabel('')  # Remove x-axis label
    axes[2, idx].set_xlabel('')  # Remove x-axis label
    axes[3, idx].set_xlabel('')  # Remove x-axis label
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [3]:
tidalphase_df

,DateTime (GMT),AllForcing,Harmonic Tide,TSI+SS,TRI+RF,TWI+WV,TSRWI,Labels,Tidal Amplitude,Station
0,1/16/2011 0:00,0.035949,-0.000303,0.026484,0.002133,0.139975,0.036253,Baseline,LT,ANN
1,1/16/2011 0:06,0.037646,-0.000783,0.028737,0.002164,0.141346,0.038428,Baseline,LT,ANN
2,1/16/2011 0:12,0.039552,-0.000978,0.030892,0.002188,0.142825,0.040530,Baseline,LT,ANN
3,1/16/2011 0:18,0.042070,-0.000897,0.033351,0.002205,0.144742,0.042967,Baseline,LT,ANN
4,1/16/2011 0:24,0.045025,-0.000533,0.035922,0.002217,0.146837,0.045558,Baseline,LT,ANN
...,...,...,...,...,...,...,...,...,...,...
83986,12/31/2011 22:36,-0.010113,0.229306,-0.271394,0.014620,-0.172478,-0.239419,Baseline,RT,LEW
83987,12/31/2011 22:42,-0.007019,0.232058,-0.270817,0.014617,-0.173068,-0.239077,Baseline,RT,LEW
83988,12/31/2011 22:48,-0.004118,0.234558,-0.270114,0.014633,-0.173609,-0.238676,Baseline,RT,LEW
83989,12/31/2011 22:54,-0.001368,0.236754,-0.269230,0.014607,-0.174066,-0.238122,Baseline,RT,LEW
